# Standard Classifier

W tym notebooku przyjrzymy się co się stanie jak użyjemy zwykłego klasyfikatora do rozwiązania naszego problemu

# Import bibliotek

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from src.dataset import *
from src.early_stopping import *
from src.utils import *

# Wczytanie danych do trenowania klasyfikatora

In [ ]:
data_train = np.load('data_transform/subset0/data.npz')
data_train

In [ ]:
X_train = data_train['images']
X_train.shape

In [ ]:
y_train = data_train['labels']
y_train.shape

# Wczytanie danych do walidacji klasyfikatora

In [ ]:
data_val = images4 = np.load('data_transform/subset4/data.npz')
data_val

In [ ]:
X_val = data_val['images']
X_val.shape

In [ ]:
y_val = data_val['labels']
y_val.shape

# Trenowanie klasyfikatora

In [ ]:
class StandardClassifier(nn.Module):
    def __init__(self, channels, no_classes, image_planes=10, images_width=32, images_height=32):
        super().__init__()
        self.model = torch.nn.Sequential()
        
        for i in range(len(channels)-1):
            self.model.add_module(f'conv_{i}', nn.Conv3d(channels[i], channels[i+1], kernel_size=3, padding=1))
            self.model.add_module(f'relu_{i}', nn.ReLU())
            self.model.add_module(f'maxpool_{i}', nn.MaxPool3d(2))

            image_planes = (image_planes - 2) // 2 + 1
            images_width = (images_width - 2) // 2 + 1
            images_height = (images_height - 2) // 2 + 1

        conv_out = channels[-1]*image_planes*images_width*images_height
            
        self.model.add_module("flatten", nn.Flatten())
        self.model.add_module("linear", nn.Linear(conv_out, conv_out))
        self.model.add_module("relu", nn.ReLU())
        self.model.add_module("dropout", nn.Dropout(0.2))
        self.model.add_module("linear_out", nn.Linear(conv_out, no_classes))
    
    def forward(self, x):
        return self.model(x)

In [ ]:
seed_everything()

train_dataset = LabelledDataset(X_train, y_train)
val_dataset = LabelledDataset(X_val, y_val)

train_dl = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=64, shuffle=True)

model = StandardClassifier([1, 16, 32, 64], 2).cuda()
optimiser = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

fit_classifier(
    model=model, optimiser=optimiser, loss_fn=loss_fn,
    train_dl=train_dl, val_dl=val_dl, epochs=50, early_stop=EarlyStopping(model_dir='model/standard', patience=5), print_metrics=True
)

# Liczenie metryk klasyfikacyjnych

In [ ]:
val_dataset = LabelledDataset(X_val, y_val)
val_dl = DataLoader(val_dataset, batch_size=64, shuffle=True)
calculate_metrics(model, val_dl)